# Importing all important libraries

In [67]:
import socket
import psutil as p
import yaml
import os
import pynvml
import threading
import speedtest
import math
# ML Specific
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neighbors import KNeighborsClassifier

# Initializing important variables

In [68]:
# Read YAML file
with open("config.yaml", 'r') as stream:
    config_loaded = yaml.safe_load(stream)
    
######################################## Connections ################################
# format: [(ip,port,connection)i] for i=onode_count
onodes_connections = []
# format: connection_object
snode_connection = None
server_connection = None
    
# Connecting to server's ip address
server_connection = socket.socket()
host = config_loaded["server"]["ip"]
port = config_loaded["server"]["port"]
o_nodes_count = config_loaded["kazaa"]["o_node_per"]

### Formats:

onodes_connections = [(ip,port,connection)i] for i=onode_count

snode_connection = connection_object

In [69]:
# Global Fields
status = ''
onodes = []
onodes_threads = [None]*o_nodes_count
snodes = []
self_kazaa_constant = 0
onodes_kazaa_constant = [0]*o_nodes_count
# this will be used to handle personal data
df = None

# Defining main functions required

In [70]:
# socket send consistent method 
# it sends the string, waits for the length of string client received, compares this length to the actual length. If both are equal
# it sends ack else, it resends the same txt again.
def socket_send(connection, txt):
    ack = False
    print(">>sending",txt+"...")
    while(not ack):
        connection.send(str.encode(txt))
        len_txt = int(connection.recv(1024).decode('utf-8'))
        if len_txt == len(txt):
            connection.send(str.encode("ack"))
            ack = True
        else:
            connection.send(str.encode("resending"))

# socket receive consistent method 
# it receives the string, then sends its size back to the server and then it waits for an acknowledgement, If ack is received it 
# breaks out of the loop
def socket_rcv(connection, size=1024):
    ack = False
    while(not ack):
        txt = connection.recv(size).decode('utf-8')
        connection.send(str.encode(str(len(txt))))
        ack = True if connection.recv(size).decode('utf-8') == "ack" else False
    print(">>received", txt+"...")
    return txt

def memory():
    mem = p.virtual_memory()
    return (mem.total, mem.available)

def cpu():
    return (100-p.cpu_percent(), p.cpu_count(logical=True))

def gpu_memory():
    try:
        pynvml.nvmlInit()
        # 1 here is the GPU id
        handle = pynvml.nvmlDeviceGetHandleByIndex(1)
        meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
        return meminfo.free
    except:
        return 0

def download_speed():
    try:
        st = speedtest.Speedtest()
        return st.download()/8
    except:
        return 0
   
def receive_rows(connection):
    print("starting receiving rows")
    row_count_str = socket_rcv(connection)
    if row_count_str.startswith("rcv:"):
        row_count = int(row_count_str[len("rcv:"):])
        headers = socket_rcv(connection)[len("rcv:"):].split(":")
        print("row_count:", row_count, "headers:", headers)
        data = []
        for i in range(row_count):
            data_rcvd = socket_rcv(connection, 32768)
            while(data_rcvd.count(":") > 3):
                socket_send(connection, "resend")
                data_rcvd = socket_rcv(connection, 32768)
            data.append(data_rcvd.split(":"))
            socket_send(connection, "rcvd")
            print("Rcvd row:", data[-1][:-1])
    return pd.DataFrame(data, columns=headers)

def send_rows_onode(connection, df, division, index, headers):
    start = division[index - 1] if index > 0 else 0
    end = division[index]
    socket_send(connection, "rcv:"+str(end - start))
    socket_send(connection, "rcv:"+headers)
    for i, row in df[start:end].iterrows():
        socket_send(connection, str(row[0])+":"+str(row[1])+":"+str(row[2])+":"+str(row[3]))
        ack = socket_rcv(connection)
        while(ack != "rcvd"):
            socket_send(connection, str(row[0])+":"+str(row[1])+":"+str(row[2])+":"+str(row[3]))
            ack = socket_rcv(connection)
            
# here snode is sender and onode is receiver
def snode_onode_socket(connection, id):
    print("Waiting for kazaa_constant of",id,"ordinary node.")
    onodes_kazaa_constant[id] = float(socket_rcv(connection)[len("k_const:"):])
        
# here onode is sender and snode is receiver
def onode_snode_socket(connection):
    socket_send(connection, "k_const:"+str(self_kazaa_constant))
    df = receive_rows(connection)
    print("df received from supernode of my region has shape: ", df.shape)
    return df
 
def send_system_info():
    messages_to_send = 5
    socket_send(server_connection, "rcv:"+str(messages_to_send))
    # download speed in bytes
    socket_send(server_connection, "dsp:"+str(download_speed()))
    # available GPU ram in bytes
    socket_send(server_connection, "gpu:"+str(gpu_memory()))
    # available ram in bytes
    socket_send(server_connection, "ram:"+str(memory()[1]))
    # cpu free in percentage
    socket_send(server_connection, "cpu:"+str(cpu()[0]))
    # cpu cores in count
    socket_send(server_connection, "cor:"+str(cpu()[1]))
    
def train_and_predict(df, df2):
    X = df[[i for i in list(df.columns)[:-1]]]
    y = df.Target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    models = {
        "knn": {"model":KNeighborsClassifier()}
    }

    for name, model in models.items():
        model['model'].fit(X_train, y_train)
        
    models_acc = []
    for name, model in models.items():
        models_acc.append(model["model"].score(X_test, y_test))
    
    for name, model in models.items():
        prediction = model['model'].predict(df2)
        
    return prediction


def most_frequent(List):
    return max(set(List), key = List.count)

print("All defined perfectly")
   

All defined perfectly


# Starting our main socket

### Sending system info

In [71]:
username = input("Input your username: ")
print('Waiting for connection response')
try:
    server_connection.connect((host, port))
    print("Connected")
except socket.error as e:
    print(str(e))
    
# first check response received i.e "Server is working"
res = socket_rcv(server_connection)
print(res)

# sending system info
send_system_info()


Input your username: ail
Waiting for connection response
Connected
>>received Server is working:...
Server is working:
>>sending rcv:5...
>>sending dsp:191531.06234391162...
>>sending gpu:0...
>>sending ram:11082366976...
>>sending cpu:82.4...
>>sending cor:4...


### Receiving acknowledgement

In [72]:
# receive acknowledge
print("\n" + "/"*40 + " waiting for ack " + "/"*40)
res = socket_rcv(server_connection)
print(res)


//////////////////////////////////////// waiting for ack ////////////////////////////////////////
>>received ack: received following system info:  
Download_speed = 191531.06234391162B 
Free_GPU_RAM = 0.0B 
Free_RAM = 11082366976.0B 
Free_CPU = 82.4 
CPU_Cores = 4...
ack: received following system info:  
Download_speed = 191531.06234391162B 
Free_GPU_RAM = 0.0B 
Free_RAM = 11082366976.0B 
Free_CPU = 82.4 
CPU_Cores = 4


### Receiving Status

In [76]:
# receive status in kazaa architecture
print("\n" + "/"*40 + " waiting for status " + "/"*40)
status = socket_rcv(server_connection)[len("status:"):]


//////////////////////////////////////// waiting for status ////////////////////////////////////////
>>received status:o...


### Receiving Kazaa Constant

In [77]:
print("\n" + "/"*40 + " waiting for kazaa constant " + "/"*40)
self_kazaa_constant = socket_rcv(server_connection)[len("k_const:"):]


//////////////////////////////////////// waiting for kazaa constant ////////////////////////////////////////
>>received k_const:0.8285163917212575...


In [78]:
print("You are appointed as " + ("Super node" if status=="s" else "Ordinary node") + " with kazaa constant as", self_kazaa_constant)

You are appointed as Ordinary node with kazaa constant as 0.8285163917212575


# Super Node

### Create a socket for ordinary nodes and send to server

In [79]:
if status == "s":
    onodesSocket = socket.socket()
    host = socket.gethostname()
    oport = port + 1
    while(True):
        try:
            onodesSocket.bind((host, oport))
            print("\n" + "/"*40 + " Supernode Socket " + "/"*40)
            print("Socket for listening to", o_nodes_count,"ordinary nodes is listening at "+host+":"+str(onodesSocket.getsockname()[1])+"\n")
            break
        except:
            oport += 1
    
    socket_send(server_connection, str(host+":"+str(onodesSocket.getsockname()[1])))
    # server_connection.send(str.encode(str(host+":"+str(onodesSocket.getsockname()[1]))))
    onodesSocket.listen(o_nodes_count)

### Accept responses from ordinary nodes and as a result get their kazaa constant

In [80]:
if status == "s":
    # open your onode socket for onodes to get connect
    id = 0
    print("\n" + "/"*40 + " waiting for kazaa constant from each ordinary node " + "/"*20)
    print("Supernode is accepting responses from ordinary nodes...")
    while(id != o_nodes_count):
        Client, address = onodesSocket.accept()
        onodes_connections.append((address[0], address[1], Client))
        onodes_threads[id] = threading.Thread(target = snode_onode_socket, args=(Client, id))
        onodes_threads[id].start()
        # print('Thread Number: ' + str(id) + " - " + 'Connected to ordinary node: ' + address[0] + ':' + str(address[1]))
        id += 1

    # joining all the threads
    for i in onodes_threads:
        i.join()

### Accept data for your region from server

In [81]:
if status == "s":
    #ensemble_distribution()
    # receive data from server
    print("\n" + "/"*40 + " waiting for region data " + "/"*40)
    df = receive_rows(server_connection)
    print("data rcvd from server for",df.shape[0],"rows with headers:",df.columns)


### Get division of data for all nodes in region

In [82]:
if status == "s":
    division = []
    for i in onodes_kazaa_constant:
        division.append(math.ceil((df.shape[0]*i)/(sum(onodes_kazaa_constant)+float(self_kazaa_constant))))
    division.append(int(df.shape[0]*float(self_kazaa_constant)/sum(onodes_kazaa_constant)))
    division

### Get headers for the data frame

In [83]:
if status == "s":
    headers = ":".join(list(df.columns))
    headers

### Send respective data to each ordinary data

In [84]:
if status == "s":
    # firstly we'll shuffle the data 
    df = df.sample(frac=1).reset_index(drop=True)
    # data for regional ordinary nodes
    index = 0
    print("\n" + "/"*40 + " waiting to send data to ordinary nodes " + "/"*40)
    print("Sending data...")
    for onode_connection in onodes_connections:
        send_rows_onode(onode_connection[2], df, division, index, headers)
        print("Sent all data to", onode_connection[0], "at port", onode_connection[1])
        index += 1

### Take out your own part of the data

In [85]:
if status == "s":
    df = df[division[-2]:division[-1]]
print("data i kept is:")
df

data i kept is:


### Get x_test from server and Predict your own results firstly

In [86]:
if status == "s":
    print("\n" + "/"*40 + " waiting to receive predictions from region " + "/"*40)
    x_test = socket_rcv(server_connection)[len("rcv_test:"):]
    predictions = []
    print("Sending x_test...")
    for onode_connection in onodes_connections:
        socket_send(onode_connection[2], "rcv_test:"+x_test)
        predictions.append(socket_rcv(onode_connection[2]))
    print(">>>>>>>>>>>", my_prediction)
    predictions.append(train_and_predict(df, pd.DataFrame([x_test.split(":")], columns=[list(df.columns)[:-1]]))[0])
    predictions.reverse()
    print(most_frequent(predictions),"is the most frequent prediction from my region.")
    socket_send(server_connection, most_frequent(predictions))

In [87]:
if status == "s":
    for i in range(248):
        print("\n" + "/"*40 + " waiting to receive predictions from region " + "/"*40)
        x_test = socket_rcv(server_connection)[len("rcv_test:"):]
        predictions = []
        print("Sending x_test...")
        for onode_connection in onodes_connections:
            socket_send(onode_connection[2], "rcv_test:"+x_test)
            predictions.append(socket_rcv(onode_connection[2]))
        my_prediction = train_and_predict(df, pd.DataFrame([x_test.split(":")], columns=[list(df.columns)[:-1]]))[0]
        print(">>>>>>>>>>>", my_prediction)
        predictions.append(my_prediction)
        predictions.reverse()
        print(most_frequent(predictions),"is the most frequent prediction from my region.")
        socket_send(server_connection, most_frequent(predictions))

# Ordinary Node

### Receiving IP:Port for super node of your current region

In [88]:
if status == "o":
    print("\n" + "/"*40 + " Ordinary Node Socket " + "/"*40)
    ip_port = socket_rcv(server_connection).split(":")
    snodes.append((ip_port[0], int(ip_port[1])))

    for ip,port in snodes:
        print("Snode of my region is present at "+ip+":"+str(port))


//////////////////////////////////////// Ordinary Node Socket ////////////////////////////////////////
>>received ubuntu:4355...
Snode of my region is present at ubuntu:4355


### After sending a connection request to the super node, ordinary nodes sends its kazaa_constant and start receiving its share in the dataset

In [89]:
if status == "o":
# starting connections with onodes:
    print("\n" + "/"*40 + " waiting to receive data from super node " + "/"*40)
    i = 0
    for ip,port in snodes:
        try:
            snode_connection = socket.socket()
            snode_connection.connect((ip, port))
            print("Connected to the supernode.\n")
            df = onode_snode_socket(snode_connection)
        except socket.error as e:
            print(str(e))
        i += 1


//////////////////////////////////////// waiting to receive data from super node ////////////////////////////////////////
Connected to the supernode.

>>sending k_const:0.8285163917212575...
starting receiving rows
>>received rcv:124...
>>received rcv:Acceleration:Speed(m/s):Angle:Target...
row_count: 124 headers: ['Acceleration', 'Speed(m/s)', 'Angle', 'Target']
>>received 43.1:26.5:14.6:Not-scored...
>>sending rcvd...
Rcvd row: ['43.1', '26.5', '14.6']
>>received 63.0:17.0:11.0:Not-scored...
>>sending rcvd...
Rcvd row: ['63.0', '17.0', '11.0']
>>received 251.0:23.0:12.6:Scored...
>>sending rcvd...
Rcvd row: ['251.0', '23.0', '12.6']
>>received 306.0:20.82:59.0:Scored...
>>sending rcvd...
Rcvd row: ['306.0', '20.82', '59.0']
>>received 133.0:30.0:66.0:Scored...
>>sending rcvd...
Rcvd row: ['133.0', '30.0', '66.0']
>>received 40.1:23.5:11.6:Not-scored...
>>sending rcvd...
Rcvd row: ['40.1', '23.5', '11.6']
>>received 123.0:29.0:31.0:Scored...
>>sending rcvd...
Rcvd row: ['123.0', '29.

### Receiving x_test from supernode of region and sending pack a prediction

In [28]:
if status == "o":
    print("\n" + "/"*40 + " waiting for x_test " + "/"*40)
    x_test = socket_rcv(snode_connection)[len("rcv_test:"):]
    df2 = pd.DataFrame([x_test.split(":")], columns=[list(df.columns)[:-1]])
    socket_send(snode_connection, train_and_predict(df, df2)[0])

In [95]:
if status == "o":
    for i in range(248):
        print("\n" + "/"*40 + " waiting for x_test " + "/"*40)
        x_test = socket_rcv(snode_connection)[len("rcv_test:"):]
        df2 = pd.DataFrame([x_test.split(":")], columns=[list(df.columns)[:-1]])
        socket_send(snode_connection, train_and_predict(df, df2)[0])


//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:186.0:28.0:32.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:187.0:29.0:33.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:188.0:30.0:34.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:189.0:31.0:35.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:190.0:32.0:36.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:191.0:33.0:37

>>received rcv_test:206.0:27.0:44.3...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:207.0:28.0:45.3...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:208.0:29.0:46.3...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:209.0:30.0:47.3...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:210.0:31.0:48.3...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:121.0:21.0:45.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ///

>>received rcv_test:134.0:31.0:67.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:135.0:32.0:68.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:136.0:33.0:69.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:137.0:34.0:70.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:138.0:35.0:71.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:139.0:36.0:72.0...
sf:: [0.88]
>>sending Scored...

//////////////////////////////////////// waiting for x_test ///

>>received rcv_test:186.0:55.6:16.8...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:187.0:56.6:17.8...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:188.0:57.6:18.8...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:87.0:16.0:17.0...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:88.0:17.0:18.0...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:89.0:18.0:19.0...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// w


//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:39.1:22.5:10.6...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:40.1:23.5:11.6...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:41.1:24.5:12.6...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:42.1:25.5:13.6...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_test:43.1:26.5:14.6...
sf:: [0.88]
>>sending Not-scored...

//////////////////////////////////////// waiting for x_test ////////////////////////////////////////
>>received rcv_tes

In [38]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [40]:
most_frequent([1,2,2])

2

In [92]:
df2

,Acceleration,Speed(m/s),Angle
0,186.0,28.0,32.0
